[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/ai-rag-lab/)


# Step 1: Setup prerequisites

In [1]:
!pip install -Uq pandas datasets

In [ ]:
import os
from pymongo import MongoClient
from utils import track_progress

import pandas as pd
pd.set_option('display.max_colwidth', 150)

from datasets import load_dataset
from typing import Dict, List

In [ ]:
# If you are using your own MongoDB Atlas cluster, use the connection string for your cluster here
MONGODB_URI = os.environ.get("MONGODB_URI")
# Initialize a MongoDB Python client
mongodb_client = MongoClient(MONGODB_URI)
# Check the connection to the server
mongodb_client.admin.command("ping")

In [ ]:
# Track progress of key steps-- DO NOT CHANGE
track_progress("cluster_creation", "primary_care_assistant_lab")

In [5]:
SERVERLESS_URL = os.environ.get("SERVERLESS_URL")

In [6]:
# Define database and collection names
DB_NAME = "virtual_primary_care_assistant"
DRUG_REVIEW_COLLECTION_NAME = "drug_reviews"

# Get a reference to the database (creates it if it doesn't exist)
db = mongodb_client[DB_NAME]

# Get a reference to collections for later use
drug_reviews_collection = db[DRUG_REVIEW_COLLECTION_NAME]

# Step 2: Load drug reviews dataset

In [7]:
# Load the drug reviews dataset from Hugging Face repository
# This dataset contains patient reviews of various medications
# 'Reboot87/drugs_reviews_dataset' contains structured data about drug experiences
drug_reviews_dataset = load_dataset(
    "Reboot87/drugs_reviews_dataset", streaming=True, split="train"
)

# Limit the dataset to 10,000 examples to manage memory usage and processing time
# This sample size should be sufficient for building our demonstration model
# The streaming=True parameter ensures we don't load the entire dataset into memory
drug_reviews_dataset = drug_reviews_dataset.take(1000)

# Similarly convert the drug reviews dataset to a DataFrame
# This enables SQL-like operations, filtering, and statistical analysis
# Having both datasets as DataFrames ensures consistent data handling approaches
drug_reviews_dataset = pd.DataFrame(drug_reviews_dataset)

# Remove the attributes patientId, date, usefulCount and review_length from the drug_reviews_dataset
# patientId: Removed to ensure data anonymization and privacy protection
# date: Temporal information isn't critical for our current analysis
# usefulCount: Engagement metrics aren't relevant for our semantic understanding
# review_length: This is a derived feature that can be recalculated if needed
drug_reviews_dataset = drug_reviews_dataset.drop(
    columns=["patientId", "date", "usefulCount", "review_length"]
)

In [ ]:
# Preview a document to understand its structure
drug_reviews_dataset.head()

In [ ]:
# Delete existing records
drug_reviews_collection.delete_many({});

# Insert new records
drug_reviews_dataset = drug_reviews_dataset.to_dict("records")

#<CODE_BLOCK_1>
drug_reviews_collection.insert_many(drug_reviews_dataset)

# Step 3: Implement search on drug reviews

### Create a search index

📚 https://www.mongodb.com/docs/atlas/atlas-search/define-field-mappings/


In [ ]:
# Define the text search index definition for the drugs_review dataset.
# This configuration specifies that only the "drugName" and "condition" fields will be indexed,
# and automatic field detection is disabled.
DRUG_REVIEW_SEARCH_INDEX_NAME='drug_review_search_index'

drug_review_text_search_model = {
    "name": DRUG_REVIEW_SEARCH_INDEX_NAME,
    "type": "search",
    "definition": {
        "analyzer": "lucene.english",
        "mappings": {
            "dynamic": False,  # Disable automatic detection; only explicitly defined fields are indexed.
            #<CODE_BLOCK_2>
            "fields": {
                "drugName": {
                    "type": "string"
                },  # Index the "drugName" field as searchable text.
                "condition": {
                    "type": "string"
                },  # Index the "condition" field as searchable text.
            }
        }
    }
}

In [ ]:
# create search index
# <CODE_BLOCK_3>
drug_reviews_collection.create_search_index(drug_review_text_search_model)

### Perform text search

📚 https://www.mongodb.com/docs/atlas/atlas-search/aggregation-stages/search/


In [ ]:
# Definition to perform text search
def drug_review_text_search(query_text):
    """
    Perform a text search in the MongoDB collection based on the user query.

    Args:
        query_text (str): The user's query string.

    Returns:
        list: A list of matching documents.
    """

    # Define the text search stage using MongoDB's $search operator.
    # This is part of Atlas Search and provides more powerful text search capabilities
    # than MongoDB's standard text index.
    # Perform a text search on the paths "drugName" and condition.
    # Cater for typo tolerance with 2 maxEdits.
    text_search_stage = {
        "$search": {
            "index": DRUG_REVIEW_SEARCH_INDEX_NAME,
            #<CODE_BLOCK_4>
            "text": {
                "query": query_text,  # The actual search term provided by the user.
                "path": ["drugName","condition"], 
                "fuzzy": {
                    "maxEdits": 2
                }
            },
        }
    }

    # Limit the number of results returned to improve performance.
    # This is especially important for large collections.
    limit_stage = {"$limit": 5}

    # Define which fields to include in the returned documents.
    # Excluding unnecessary fields reduces bandwidth and processing overhead.
    project_stage = {
        "$project": {
            "_id": 0,  # Exclude MongoDB's internal ID field.
            "embedding": 0,  # Exclude the embedding field.
        }
    }

    # Combine all stages into a MongoDB aggregation pipeline.
    # The pipeline will execute stages in sequence: search, limit, then project.
    pipeline = [text_search_stage, limit_stage, project_stage]

    # Execute the search by running the aggregation pipeline against the specified collection.
    # Convert the cursor to a list to ensure results are fully fetched before the function returns.
    #results = <CODE_BLOCK_5>
    results = drug_reviews_collection.aggregate(pipeline)

    return list(results)

In [ ]:
temp_results = drug_review_text_search("I am coughing")
pd.DataFrame(temp_results).head()

In [ ]:
# test our text search by mispelling dextromethorphan
temp_results = drug_review_text_search("How is dextormethorphan")

# notice the false positives
pd.DataFrame(temp_results).head()

# Step 4: Generate embeddings on drug reviews

In [15]:
# You may see a warning upon running this cell. You can ignore it.
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [16]:
# Load the `gte-small` model using the Sentence Transformers library
embedding_model = SentenceTransformer("thenlper/gte-small")

📚 https://huggingface.co/thenlper/gte-small#usage (See "Use with sentence-transformers" under Usage)

In [ ]:
# Define a function that takes a piece of text (`text`) as input, embeds it using the `embedding_model` instantiated above and returns the embedding as a list
# An array can be converted to a list using the `tolist()` method
def get_embedding(text: str) -> List[float]:
    """
    Generate the embedding for a piece of text.

    Args:
        text (str): Text to embed.

    Returns:
        List[float]: Embedding of the text as a list.
    """
    #embedding = <CODE_BLOCK_6>
    embedding = embedding_model.encode(text)
    return embedding.tolist()

In [ ]:
embedded_docs = []
# Add an `embedding` field to each dictionary in `drug_reviews_dataset`
# The `embedding` field should correspond to the embedding of the value of the `review` field
# Use the `get_embedding` function defined above to generate the embedding
# Append the updated dictionaries to `embedded_docs` initialized above.
for doc in tqdm(drug_reviews_dataset):

    # format doc to proper sentence
    record = (
        f"Drug Name: {doc['drugName']}. "
        f"Condition: {doc['condition']}. "
        f"Review: {doc['review']}."
    )
    # <CODE_BLOCK_7>
    doc['embedding'] = get_embedding(record)
    embedded_docs.append(doc)

In [ ]:
# Check that the length of `embedded_docs` should be same as the dataset
len(embedded_docs)

# Step 5: Re-ingest data into MongoDB


In [ ]:
drug_reviews_collection.delete_many({})

In [ ]:
# Bulk insert `embedded_docs` into the `collection` defined above -- should be a one-liner
drug_reviews_collection.insert_many(embedded_docs)
print(f"Ingested {drug_reviews_collection.count_documents({})} documents into the {DRUG_REVIEW_COLLECTION_NAME} collection.")

# Step 6: Implement vector search capability

### Create a vector search index

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-type/


In [ ]:
# Create vector index definition specifying:
# path: Path to the embeddings field
# numDimensions: Number of embedding dimensions- depends on the embedding model used
# similarity: Similarity metric. One of cosine, euclidean, dotProduct.
DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME='drug_review_vector_search_index'

drug_review_vector_search_model = {
    "name": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,
    "type": "vectorSearch",
    "definition": {
        "fields": [
            # Define a vector field on the path "embedding"
            # It has 384 dimensions and uses the cosine similarity
            # <CODE_BLOCK_8>
            {
                "type": "vector",
                "path": "embedding",
                "numDimensions": 384,
                "similarity": "cosine",
            }
        ]
    },
}

In [ ]:
# create vector search index
# <CODE_BLOCK_9>
drug_reviews_collection.create_search_index(drug_review_vector_search_model)

In [ ]:
# Use the `check_index_ready` function from the `utils` module to verify that the index was created and is in READY status before proceeding
from utils import check_index_ready
check_index_ready(drug_reviews_collection, DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME)

In [ ]:
# Track progress of key steps-- DO NOT CHANGE
track_progress("vs_index_creation", "primary_care_assistant_lab")

### Define a vector search function

📚 https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-stage/#ann-examples (Refer to the "Basic Example")


In [ ]:
# Define a function to retrieve relevant documents for a user query using vector search
def drug_review_vector_search(user_query: str) -> List[Dict]:
    """
    Retrieve relevant documents for a user query using vector search.

    Args:
    user_query (str): The user's query string.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the `user_query` using the `get_embedding` function defined in Step 4
    query_embedding = get_embedding(user_query)

    # Define an aggregation pipeline consisting of a $vectorSearch stage, followed by a $project stage
    # Set the number of candidates to 50 and only return the top 5 documents from the vector search
    # NOTE: Use variables defined previously for the `index`, `queryVector` and `path` fields in the $vectorSearch stage
    pipeline = [
    {
        #vector search stage
        #<CODE_BLOCK_10>
        "$vectorSearch": {
            "index": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 50,
            "limit": 5
        }
    },
    {
        "$project": {
            "_id": 0,
            "embedding": 0,
            "score": {"$meta": "vectorSearchScore"}
        }
    }
]

    # Execute the aggregation `pipeline` and store the results in `results`
    results = drug_reviews_collection.aggregate(pipeline)
    return list(results)

### Run vector search queries


In [ ]:
temp_results = drug_review_vector_search("How is dextromethorphan")

pd.DataFrame(temp_results).head()

# Step 7: Implementing hybrid search

In [ ]:
def hybrid_search(
    text_search_query,
    vector_search_query,
    vector_weight=0.5,
    full_text_weight=0.5,
    top_k=10,
    text_search_paths=["drugName","condition"],
):
    """
    Conduct a hybrid search on a MongoDB Atlas collection that combines a vector search
    and a full-text search using Atlas Search.

    Args:
        user_query (str): The user's query string.
        collection (MongoCollection): The MongoDB collection to search.
        vector_search_index_name (str): The name of the vector search index.
        text_search_index_name (str): The name of the text search index.
        vector_weight (float): The weight of the vector search.
        full_text_weight (float): The weight of the full-text search.
        top_k (int): Number of results to return.

    Returns:
        list: A list of documents (dict) with combined scores.
    """

    # Get the pre-computed embedding vector for the user's query
    query_vector = get_embedding(vector_search_query)

    # Create a MongoDB aggregation pipeline to perform hybrid search
    pipeline = [
        # PART 1: VECTOR SEARCH
        # Perform semantic vector search using the query embedding
        {
            "$vectorSearch": {
                "index": DRUG_REVIEW_VECTOR_SEARCH_INDEX_NAME,  # Name of the vector search index
                "path": "embedding",  # Field containing document embeddings
                "queryVector": query_vector,  # The query vector to compare against
                "numCandidates": top_k*10,  # Number of candidates to consider for similarity
                "limit": top_k,  # Initial limit of results
            }
        },
        # Group all vector search results into a single document
        # This prepares for the ranking step
        {
            "$group": {
                "_id": None,
                "docs": {"$push": "$$ROOT"},  # Push all documents into an array
            }
        },
        # Unwind the array of documents to process each individually
        # This adds a rank based on the original vector search order
        {
            "$unwind": {
                "path": "$docs",
                "includeArrayIndex": "rank",  # Add the array index as a rank field
            }
        },
        # Calculate a vector search score based on rank
        # Higher ranks get lower scores via division formula
        {
            "$addFields": {
                "vs_score": {
                    "$multiply": [
                        vector_weight,  # Apply configurable weight to vector scores
                        {
                            "$divide": [1.0, {"$add": ["$rank", 60]}]
                        },  # Score formula: 1/(rank+60)
                    ]
                }
            }
        },
        # Project only the needed fields from each document
        # Including the calculated vector search score
        {
            "$project": {
                "vs_score": 1,
                "_id": "$docs._id",
                "review": "$docs.review",
                "drugName": "$docs.drugName",
                "condition": "$docs.condition",
            }
        },
        # PART 2: TEXT SEARCH
        # Combine with full-text search results using unionWith
        {
            "$unionWith": {
                "coll": DRUG_REVIEW_COLLECTION_NAME,  # Collection to search
                "pipeline": [
                    # Perform full text search using Atlas Search
                    {
                        "$search": {
                            "index": DRUG_REVIEW_SEARCH_INDEX_NAME,  # Name of the text search index
                            "text": {
                                "query": text_search_query,  # Raw text query from user
                                "path": text_search_paths,  # Field to search in
                                "fuzzy": {
                                     "maxEdits": 2
                                }
                            },
                        }
                    },
                    {"$limit": top_k},  # Limit initial text search results
                    # Group text search results similar to vector search
                    {"$group": {"_id": None, "docs": {"$push": "$$ROOT"}}},
                    # Unwind and add ranking just like in vector search
                    {"$unwind": {"path": "$docs", "includeArrayIndex": "rank"}},
                    # Calculate a full-text search score based on rank
                    # Using the same formula as vector search
                    {
                        "$addFields": {
                            "fts_score": {
                                "$multiply": [
                                    full_text_weight,  # Apply configurable weight to text scores
                                    {"$divide": [1.0, {"$add": ["$rank", 60]}]},
                                ]
                            }
                        }
                    },
                    # Project only the needed fields for text search results
                    {
                        "$project": {
                            "fts_score": 1,
                            "_id": "$docs._id",
                            "review": "$docs.review",
                            "drugName": "$docs.drugName",
                            "condition": "$docs.condition",
                        }
                    },
                ],
            }
        },
        # PART 3: COMBINING RESULTS
        # Group by document ID to handle duplicates from both searches
        # This ensures we don't return the same document twice
        {
            "$group": {
                "_id": "$_id",
                "review": {"$first": "$review"},
                "drugName": {"$first": "$drugName"},
                "condition": {"$first": "$condition"},
                "vs_score": {
                    "$max": "$vs_score"
                },  # Take highest vector score if present in both
                "fts_score": {
                    "$max": "$fts_score"
                },  # Take highest text score if present in both
            }
        },
        # Handle documents that only appeared in one search type
        # by setting missing scores to 0
        {
            "$project": {
                "_id": 1,
                "review": 1,
                "drugName": 1,
                "condition": 1,
                "vs_score": {
                    "$ifNull": ["$vs_score", 0]
                },  # Default to 0 if not in vector results
                "fts_score": {
                    "$ifNull": ["$fts_score", 0]
                },  # Default to 0 if not in text results
            }
        },
        # Calculate the final combined score and remove _id from results
        {
            "$project": {
                "score": {"$add": ["$fts_score", "$vs_score"]},  # Combined final score
                "_id": 0,  # Exclude MongoDB ID
                "review": 1,
                "drugName": 1,
                "condition": 1,
                "vs_score": 1,  # Keep individual scores for analysis
                "fts_score": 1,
            }
        },
        # Sort by the combined score in descending order
        {"$sort": {"score": -1}},
        # Return only the top k results based on combined score
        {"$limit": top_k},
    ]

    # Execute the aggregation pipeline and convert results to a list
    results = list(drug_reviews_collection.aggregate(pipeline))
    return results

In [ ]:
# query_text = "I have a cough, what drug would be best?"
query_text = "How is dextromethorphan?"

# Execute a hybrid search that combines both vector (semantic) and full-text search
# We can change the weightage if necessary

drug_reviews_hybrid_results = hybrid_search(
    query_text,  # Our natural language query
    query_text
)

pd.DataFrame(drug_reviews_hybrid_results).head()

# Step 8: Build the RAG application


In [32]:
import requests

### Define a function to create the chat prompt

In [33]:
def format_context(context) -> str:
    formatted_context = ""

    # Check if any documents were retrieved.
    if context and len(context) > 0:
        # Add a header for the context section.
        formatted_context = "\n\nRelevant information from drug reviews:\n\n"

        # Process each retrieved document and format its content.
        for i, doc in enumerate(context):
            # Extract key fields from the document.
            review = doc.get("review", "No review available")
            condition = doc.get("condition", "No condition available")
            drug_name = doc.get("drugName", "No drug name available")

            # Append the formatted document with a citation reference.
            formatted_context += f"[{i+1}] Review: {review}\nCondition: {condition}\nDrug Name: {drug_name}\n\n"
    return formatted_context

In [ ]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = hybrid_search(user_query,user_query)
    # 1. Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
    # 2. Format the retrieved documents into context for the LLM.
    #formatted_context = <CODE_BLOCK_11>
    formatted_context = format_context(context)

    # 3. Craft the prompt for the LLM using the user query and the formatted context.
    prompt = f"""
Based on the following information, please answer the user's question:
User Question: {user_query}
{formatted_context}
Please provide a comprehensive answer based on the information above.
If the provided information does not contain the answer, state that clearly.
Include citation numbers [X] to indicate which sources were used for specific details.
"""

    # Prompt consisting of the question and relevant context to answer it
    return prompt

### Define a function to answer user queries

In [ ]:
# Define a function to answer user queries
def generate_answer(user_query: str) -> None:
    """
    Generate an answer to the user query.

    Args:
        user_query (str): The user's query string.
    """
    # Use the `create_prompt` function above to create a chat prompt
    prompt = create_prompt(user_query)
    # Format the message to the LLM in the format [{"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # Use the `prompt` created above to populate the `content` field in the chat message
    #messages = <CODE_BLOCK_12>
    messages = [{"role":"user","content":prompt}]
    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})
    # Print the final answer
    print(response.json()["text"])

### Query the RAG application


In [ ]:
query_text = "I have a cough, what drug would be best?"
#query_text = "How is dextromethorphan?"

generate_answer(query_text)

# 🦹‍♀️ "Agentic" Pipeline


In [45]:
# Use LLM to filter out terms that are medical conditions, symptoms, or drug names
def get_terms_from_query(user_query: str) -> str:
    prompt = f"""
        For the following query, return me only words associated with medical conditions, symptoms or drug names: {user_query}.
        Respond only with the words, space separated.
        """

    messages = [{"role":"user","content":prompt}]
    # Send the chat messages to a serverless function to get back an LLM response
    
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})
    return response.json()["text"]

In [ ]:
# test the definition
get_terms_from_query("I have cough and sore throat")

In [47]:
def get_medication_reviews(user_query:str) -> str:
    """
    Retrieves patient reviews and information about medications related to the query.

    This tool searches a database of medication reviews to find relevant patient experiences
    with drugs that match the symptoms, conditions, or medication names in the user query.
    Use this tool when discussing specific medications or treatment options.

    Args:
        user_query (str): The medication name, condition, or symptom to search for reviews about.

    Returns:
        str: Patient reviews and experiences with relevant medications.
    """
    
    refined_text_search_query = get_terms_from_query(user_query)
    
   # Execute the hybrid search to find medication reviews
    retrieved_context = hybrid_search(
        refined_text_search_query,
        user_query,
    )

    return retrieved_context

In [ ]:
temp_results = get_medication_reviews("I have a cought and sore throat, what is best for me")

pd.DataFrame(temp_results).head()

In [50]:
# Define a function to create the user prompt for our RAG application
def create_prompt(user_query: str) -> str:
    """
    Create a chat prompt that includes the user query and retrieved context.

    Args:
        user_query (str): The user's query string.

    Returns:
        str: The chat prompt string.
    """
    # Retrieve the most relevant documents for the `user_query` using the `vector_search` function defined in Step 7
    context = get_medication_reviews(user_query)
    # Join the retrieved documents into a single string, where each document is separated by two new lines ("\n\n")
        # 2. Format the retrieved documents into context for the LLM.
    formatted_context = format_context(context)

    # 3. Craft the prompt for the LLM using the user query and the formatted context.
    prompt = f"""
Based on the following information, please answer the user's question:
User Question: {user_query}
{formatted_context}
Please provide a comprehensive answer based on the information above.
If the provided information does not contain the answer, state that clearly.
Include citation numbers [X] to indicate which sources were used for specific details.
"""

    # Prompt consisting of the question and relevant context to answer it
    return prompt

In [ ]:
# Use this if you have not completed "adding memory"
# generate_answer("I have a flu")

# Use this if you have completed "adding memory"
generate_answer("session", "I have a flu")

# Step 9: Add memory to the RAG application


In [37]:
from datetime import datetime

In [38]:
history_collection = mongodb_client[DB_NAME]["chat_history"]

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.create_index


In [ ]:
# Create an index on the key `session_id` for the `history_collection` collection
history_collection.create_index("session_id")

### Define a function to store chat messages in MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/collection.html#pymongo.collection.Collection.insert_one

In [40]:
def store_chat_message(session_id: str, role: str, content: str) -> None:
    """
    Store a chat message in a MongoDB collection.

    Args:
        session_id (str): Session ID of the message.
        role (str): Role for the message. One of `system`, `user` or `assistant`.
        content (str): Content of the message.
    """
    # Create a message object with `session_id`, `role`, `content` and `timestamp` fields
    # `timestamp` should be set the current timestamp
    message = {
        "session_id": session_id,
        "role": role,
        "content": content,
        "timestamp": datetime.now(),
    }
    # Insert the `message` into the `history_collection` collection
    history_collection.insert_one(message)

### Define a function to retrieve chat history from MongoDB

📚 https://pymongo.readthedocs.io/en/stable/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort

In [41]:
def retrieve_session_history(session_id: str) -> List:
    """
    Retrieve chat message history for a particular session.

    Args:
        session_id (str): Session ID to retrieve chat message history for.

    Returns:
        List: List of chat messages.
    """
    # Query the `history_collection` collection for documents where the "session_id" field has the value of the input `session_id`
    # Sort the results in increasing order of the values in `timestamp` field
    cursor = history_collection.find({"session_id": session_id}).sort("timestamp", 1)

    if cursor:
        # Iterate through the cursor and extract the `role` and `content` field from each entry
        # Then format each entry as: {"role": <role_value>, "content": <content_value>}
        messages = [{"role": msg["role"], "content": msg["content"]} for msg in cursor]
    else:
        # If cursor is empty, return an empty list
        messages = []

    return messages

### Handle chat history in the `generate_answer` function

In [42]:
def generate_answer(session_id: str, user_query: str) -> None:
    """
    Generate an answer to the user's query taking chat history into account.

    Args:
        session_id (str): Session ID to retrieve chat history for.
        user_query (str): The user's query string.
    """
    # Initialize list of messages to pass to the chat completion model
    messages = []

    # Retrieve documents relevant to the user query and convert them to a single string
    prompt = create_prompt(user_query)
    # Create a system prompt containing the retrieved context
    system_message = {
        "role": "user",
        "content": prompt
    }
    # Append the system prompt to the `messages` list
    messages.append(system_message)

    # Use the `retrieve_session_history` function to retrieve message history from MongoDB for the session ID `session_id` 
    # And add all messages in the message history to the `messages` list 
    message_history = retrieve_session_history(session_id)
    messages.extend(message_history)

    # Format the user query in the format {"role": <role_value>, "content": <content_value>}
    # The role value for user messages must be "user"
    # And append the user message to the `messages` list
    user_message = {"role": "user", "content": user_query}
    messages.append(user_message)

    # Send the chat messages to a serverless function to get back an LLM response
    response = requests.post(url=SERVERLESS_URL, json={"task": "completion", "data": messages})

    # Extract the answer from the response
    answer = response.json()["text"]

    # Use the `store_chat_message` function to store the user message and also the generated answer in the message history collection
    # The role value for user messages is "user", and "assistant" for the generated answer
    store_chat_message(session_id, "user", user_query)
    store_chat_message(session_id, "assistant", answer)

    print(answer)

In [ ]:
generate_answer(
    session_id="1",
    user_query="I have a sore throat",
)

In [ ]:
generate_answer(
    session_id="1",
    user_query="What if i have a cough too",
)